In [1]:
import os
import uuid
import dotenv
from IPython import display

from langchain import prompts
from langchain_core import messages
from langchain_core import runnables

import langchain_neo4j
import langchain_ollama
import langchain_openai
import langchain_google_genai

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import cypher_mod

dotenv.load_dotenv(".env")

True

In [ ]:
# Tambahkan perintah jika tidak tahu, maka tambahkan "UTRD" di awal kalimat

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.

Follow this instruction when generating answers.
- Don't answer the user question if it not in legal/law scope, like science, math, social, ect.
- If the provided information is empty, say "Tidak dapat menemukan data yang sesuai dengan permintaan query".

Provided Information:
{context}

Question:
{question}

Helpful Answer:"""

CYPHER_QA_PROMPT = prompts.PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

# Follow this example when generating answers.
# If the provided information is empty, say that you don't know the answer and start with the text "UTRD".
# GraphCypherQASchema(answer="Tidak dapat menemukan data berdasarkan permintaan query", is_query_answered=False)

In [3]:
# Aku tambahkan contoh kode cypher nya

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Following are some examples that you can use as a reference to create Cypher code according to user questions.

User query   : Apa isi pasal 100 UU nomor 90 tahun 2020?
Cypher query : MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'UU' AND r.number = 90 AND r.year = 2020 AND a.number = '100'
RETURN a.text AS text

User query   : Apa isi pasal selanjutnya dari pasal 100 undang-undang / UU nomor 90 tahun 2020?
Cypher query : MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'UU' AND r.number = 90 AND r.year = 2020 AND a.number = '100'
RETURN next_article.text AS text

Don't use `real_text` attribute! But use `text` attribute instead.

Now, make a cypher code for the following user questions.

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = prompts.PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
graph = langchain_neo4j.Neo4jGraph(
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_SMALL"],
    enhanced_schema=True
)

cypher_llm = langchain_google_genai.ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

qa_llm = langchain_google_genai.ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

# cypher_llm = langchain_ollama.ChatOllama(model="qwen2.5-coder:7b-instruct-q4_K_M", temperature=0.0)
# qa_llm = langchain_ollama.ChatOllama(model="qwen2.5:7b-instruct-q4_K_M", temperature=0.0)
# qa_llm = langchain_ollama.ChatOllama(model="llama3.2:3b-instruct-q4_K_M", temperature=0.3)
# qa_llm = langchain_ollama.ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.0)

# Ubah-ubah nilai variabel ini
return_direct = False

def text2cypher(query: str) -> str:
    
    # Initialize the Neo4j Graph QA Chain
    text2cypher_chain = cypher_mod.GraphCypherQAChainMod.from_llm(
        # llm=llm,
        graph=graph,
        qa_prompt=CYPHER_QA_PROMPT,
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        cypher_llm=cypher_llm,
        qa_llm=qa_llm,
        exclude_types=["embedding"],
        include_types=[],
        return_intermediate_steps=not return_direct,
        return_direct=return_direct,
        allow_dangerous_requests=True,
        verbose=True
    )

    result = text2cypher_chain.invoke(query)

    if not result["cypher"]:
        print("cypher None")
        result["cypher"] = messages.AIMessage(
            content="Tidak dapat membuat kode Cypher Neo4j berdasarkan permintaan query"
        )

    if return_direct:
        artifact = {"is_context_fetched": bool(result["result"])}

        if not result["result"]:
            print(f"return_direct=True, but result=[]")
            result["result"] = ["Tidak dapat menemukan data yang sesuai dengan permintaan query"]

        response = (
            "### **Hasil Pembuatan Kode Cypher:**\n"
            f"{result['cypher'].content}\n\n"
            "### **Hasil Eksekusi Kode Cypher ke Database:**\n"
            f"{result['result']}"
        )
    
    else:
        artifact = {"is_context_fetched": bool(result["context"])}

        if hasattr(result["result"], "content") and result["result"].content == "":
            print(f"return_direct=False, but result=\"\"")
            result["result"] = messages.AIMessage(
                content="Tidak dapat menemukan data yang sesuai dengan permintaan query"
            )

        response = (
            "### **Hasil Pembuatan Kode Cypher:**\n"
            f"{result['cypher'].content}\n\n"
            "### **Hasil Eksekusi Kode Cypher ke Database:**\n"
            f"{result['result'].content}"
        )

    # return result
    return response, artifact

In [5]:
response = text2cypher("Apa isi pasal 28 UU No. 11 tahun 2008?")
# response = text2cypher("Apa isi pasal 100 UU No 11 tahun 2008?")
# response = text2cypher("Kenapa ikan berenang di air?")



> Entering new GraphCypherQAChainMod chain...
Generated Cypher:
cypher
MATCH (r:Regulation {type: 'UU', number: 11, year: 2008})
  -[HAS_ARTICLE]->(a:Article {number: "28"})
RETURN a.text

Return Direct:
False

Full Context:
[{'a.text': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 28:\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).'}]

> Finished chain.


In [6]:
display.display(display.Markdown(response[0]))
display.display(response[1])

### **Hasil Pembuatan Kode Cypher:**
```cypher
MATCH (r:Regulation {type: 'UU', number: 11, year: 2008})
  -[HAS_ARTICLE]->(a:Article {number: "28"})
RETURN a.text
```

### **Hasil Eksekusi Kode Cypher ke Database:**
Pasal 28 UU No. 11 Tahun 2008 mengatur tentang:

(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.

(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).

{'is_context_fetched': True}